### 180

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [2]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=1)
# sys.exit()
df_hist = utils.read_df_pkl('../input/hist_clean_rdm0*')
df_hist = reduce_mem_usage(df_hist)

df_org_lag = utils.read_pkl_gzip('../input/train_test_original_month_lag.gz')
df_org_lag = reduce_mem_usage(df_org_lag)
df_org_lag.head()

df_org_lag.rename(columns={'purchase_date':'yyyymm'}, inplace=True)
print(df_hist.shape)
df_hist = df_hist.merge(df_org_lag.reset_index()[[key, 'yyyymm', 'org_month_lag']], how='inner', on=[key, 'yyyymm'])
print(df_hist.shape)

100%|██████████| 3/3 [00:15<00:00,  5.07s/it]


Mem. usage decreased to 2554.26 Mb (0.0% reduction)
Mem. usage decreased to 72.58 Mb (21.9% reduction)
(29112361, 21)
(29112361, 22)


In [4]:
df = df_hist
auth1 = df[df.authorized_flag==1]
auth0 = df[df.authorized_flag==0]

### New Merchant

In [5]:
m_id = 'merchant_id'
mc_id = 'merchant_category_id'
lag_col = 'org_month_lag'
lag_list = list(reversed(sorted(list(auth1[lag_col].value_counts().index))))
lag_list[:10]

[75, 74, 73, 72, 71, 70, 69, 68, 67, 66]

In [9]:
feat_list = []
# lag1_list = np.arange(6) + 1
lag1_list = (np.zeros(6)).astype('int8')
lag2_list = np.arange(6) + 3
lag1_list = [3]
lag2_list = [4]
     
fname_list = [ f'auth1_new_' ,f'auth0_new_']
    
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

    
def feat_agg(df, fname):
# for df, fname in zip(df_list, fname_list):
    global train_test
    
#     aggs = {}
    if fname.count('cat1'):
        df = pd.get_dummies(df, columns=['category_3'])
        aggs = {
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        }
    else:
        df = pd.get_dummies(df, columns=['category_2', 'category_3'])
        aggs = {
        'category_2_1.0': ['mean'],
        'category_2_2.0': ['mean'],
        'category_2_3.0': ['mean'],
        'category_2_4.0': ['mean'],
        'category_2_5.0': ['mean'],
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        }
    
    if not(fname.count('cat')):
        aggs['category_1'] = ['sum', 'mean']
            
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['first_active_month'] =  pd.to_datetime(df['first_active_month'])
    df['month_diff'] = (pd.to_datetime('2018-05-01') - df['purchase_date']).dt.days
    
    aggs['month_lag'] = ['mean', 'std']
    aggs['yyyy_week'] = ['nunique']
    aggs['purchase_date'] = ['max','min']
    aggs['month_diff'] = ['mean', 'std']
    
    aggs['purchase_amount'] = ['sum','max', 'min','mean']
    aggs['installments'] = ['sum', 'max', 'min','mean', 'std']
#     aggs['purchase_amount_over'] = ['sum','max', 'min','mean']
    
    aggs['merchant_id'] = ['nunique']
    aggs['merchant_category_id'] = ['nunique']
    aggs['card_id'] = ['size']
    aggs['city_id'] = ['nunique']
    aggs['state_id'] = ['nunique']
    aggs['subsector_id'] = ['nunique']
#     aggs['all_term'] = ['max']
    
    new_columns = get_new_columns(fname, aggs)
    df_agg = df.groupby(key).agg(aggs)
    df_agg.columns = new_columns
    df_agg.reset_index(drop=False,inplace=True)
     
    df_agg[f'{fname}_this_term'] =  (pd.to_datetime(df_agg[f'{fname}_purchase_date_max']) - pd.to_datetime(df_agg[f'{fname}_purchase_date_min'])).dt.days
    df_agg[f'{fname}_freq_per_this_term'] = df_agg[f'{fname}_card_id_size']              / (df_agg[f'{fname}_this_term'] + 1.0)
    df_agg[f'{fname}_amount_per_this_term'] = df_agg[f'{fname}_purchase_amount_sum']     / (df_agg[f'{fname}_this_term'] + 1.0)
    df_agg[f'{fname}_instthisments_per_this_term'] = df_agg[f'{fname}_installments_sum'] / (df_agg[f'{fname}_this_term'] + 1.0)
    
    df_agg[f'{fname}_freq_per_90days'] = df_agg[f'{fname}_card_id_size'] / 90
    df_agg[f'{fname}_amount_per_90days'] = df_agg[f'{fname}_purchase_amount_sum'] / 90
    df_agg[f'{fname}_installments_per_90days'] = df_agg[f'{fname}_installments_sum'] / 90
    
    df_agg[f'{fname}_amount_per_installments_sum'] = df_agg[f'{fname}_purchase_amount_sum'] / (df_agg[f'{fname}_installments_sum'] + 1.0)
    df_agg[f'{fname}_amount_per_installments_mean'] = df_agg[f'{fname}_purchase_amount_mean'] / (df_agg[f'{fname}_installments_mean'] + 1.0)
    
#     df_agg.drop(f'{fname}_all_term_max', axis=1, inplace=True)
    
    # Monthly Agg
#     del aggs['all_term']
    new_columns = get_new_columns(fname + '_monthly_avg', aggs)
    month_agg = df.groupby([key, 'yyyymm']).agg(aggs)
        
    month_agg.columns = new_columns
    month_agg = month_agg.reset_index().drop('yyyymm', axis=1).groupby([key]).mean()
    month_agg.reset_index(drop=False,inplace=True)
    
    df_feat = train_test.join(df_agg.set_index(key)).join(month_agg.set_index(key))
    print(fname)
    print(df_feat.shape)
    display(df_feat.head())
    
    del df_agg, month_agg
    gc.collect()
    
    feat_list.append(df_feat)
    
    
for df, fname in zip([auth1, auth0], fname_list):

    for num, (lag1, lag2) in enumerate(zip(lag1_list, lag2_list)):
        
        fname += f'lag{lag1}_{lag2}'
        
        df_train = utils.read_df_pkl('../input/train0*')
        df_test = utils.read_df_pkl('../input/test0*')
        df_train.set_index(key, inplace=True)
        df_test.set_index(key, inplace=True)
        train_test = pd.concat([df_train, df_test], axis=1)
        train_test.drop(['feature_1', 'feature_2', 'feature_3', 'first_active_month', 'target'], axis=1, inplace=True)
        
        tmp = df[df[lag_col]>lag2]
        now = df[df[lag_col]<=lag2]
        now = now[now[lag_col]>=lag1]
        
        df_now = now[[key, mc_id, m_id]].drop_duplicates()
        df_mer = tmp[[key, mc_id, m_id]].drop_duplicates()
        
        # inner joinで過去に存在したmerchantをdf_nowに残し、そのindexをdf_nowから除外することでnew_merchantのみ残す。天才！
        df_now['index'] = np.arange(len(df_now))
        mer_same = df_now.merge(df_mer, how='inner', on=[key, m_id, mc_id])
        past_mer_index = mer_same['index'].values
        df_new = df_now.loc[~df_now['index'].isin(past_mer_index), :]
        
        df_new.drop('index', axis=1, inplace=True)
        df_new.dropna(inplace=True)
        
        # ここで、new_merchantに対する購買履歴のみのテーブルができる
        new_hist = df_new.merge(df, how='inner', on=[key, mc_id, m_id])
        print(new_hist.shape)
        
        feat_agg(new_hist, fname)

#========================================================================
# impute
#========================================================================

def impute_agg(df):
    for col in df.columns:
        if col.count('category_1'):
            df[col].fillna(-1, inplace=True)
        if col.count('isnull'):
            df[col].fillna(-1, inplace=True)
        if col.count('null_cnt'):
            df[col].fillna(-1, inplace=True)
        if col.count('month_lag'):
            df[col].fillna(df[col].min()-15, inplace=True)
        if col.count('month_diff'):
            df[col].fillna(df[col].max()+100, inplace=True)
        if col.count('nunique'):
            df[col].fillna(-1, inplace=True)
        if col.count('amount'):
            df[col].fillna(-1, inplace=True)
        if col.count('installments'):
            df[col].fillna(-2, inplace=True)
        if col.count('unix_date'):
            df[col].fillna(df[col].min()-100000, inplace=True)
        if col.count('size'):
            df[col].fillna(-1, inplace=True)
        if col.count('term'):
            df[col].fillna(-1, inplace=True)
        if col.count('per'):
            df[col].fillna(-1, inplace=True)
        if col.count('dummie'):
            df[col].fillna(-1, inplace=True)
            
    return df

        
for df in feat_list:
    try:
        df.drop(['feature_1', 'feature_2', 'feature_3', 'first_active_month', 'target'], axis=1, inplace=True)
    except KeyError:
        pass
    
    try:
        df.set_index(key, inplace=True)
    except KeyError:
        pass
    
    train_test = train_test.join(df)

train_test = impute_agg(train_test)

display(train_test.head())

100%|██████████| 3/3 [00:00<00:00, 95.16it/s]


(3480192, 22)
auth1_new_lag3_4
(325540, 71)


,auth1_new_lag3_4_category_2_1.0_mean,auth1_new_lag3_4_category_2_2.0_mean,auth1_new_lag3_4_category_2_3.0_mean,auth1_new_lag3_4_category_2_4.0_mean,auth1_new_lag3_4_category_2_5.0_mean,auth1_new_lag3_4_category_3_A_mean,auth1_new_lag3_4_category_3_B_mean,auth1_new_lag3_4_category_3_C_mean,auth1_new_lag3_4_category_1_sum,auth1_new_lag3_4_category_1_mean,auth1_new_lag3_4_month_lag_mean,auth1_new_lag3_4_month_lag_std,auth1_new_lag3_4_yyyy_week_nunique,auth1_new_lag3_4_purchase_date_max,auth1_new_lag3_4_purchase_date_min,auth1_new_lag3_4_month_diff_mean,auth1_new_lag3_4_month_diff_std,auth1_new_lag3_4_purchase_amount_sum,auth1_new_lag3_4_purchase_amount_max,auth1_new_lag3_4_purchase_amount_min,auth1_new_lag3_4_purchase_amount_mean,auth1_new_lag3_4_installments_sum,auth1_new_lag3_4_installments_max,auth1_new_lag3_4_installments_min,auth1_new_lag3_4_installments_mean,auth1_new_lag3_4_installments_std,auth1_new_lag3_4_merchant_id_nunique,auth1_new_lag3_4_merchant_category_id_nunique,auth1_new_lag3_4_card_id_size,auth1_new_lag3_4_city_id_nunique,auth1_new_lag3_4_state_id_nunique,auth1_new_lag3_4_subsector_id_nunique,auth1_new_lag3_4_this_term,auth1_new_lag3_4_freq_per_this_term,auth1_new_lag3_4_amount_per_this_term,auth1_new_lag3_4_instthisments_per_this_term,auth1_new_lag3_4_freq_per_90days,auth1_new_lag3_4_amount_per_90days,auth1_new_lag3_4_installments_per_90days,auth1_new_lag3_4_amount_per_installments_sum,auth1_new_lag3_4_amount_per_installments_mean,auth1_new_lag3_4_monthly_avg_category_2_1.0_mean,auth1_new_lag3_4_monthly_avg_category_2_2.0_mean,auth1_new_lag3_4_monthly_avg_category_2_3.0_mean,auth1_new_lag3_4_monthly_avg_category_2_4.0_mean,auth1_new_lag3_4_monthly_avg_category_2_5.0_mean,auth1_new_lag3_4_monthly_avg_category_3_A_mean,auth1_new_lag3_4_monthly_avg_category_3_B_mean,auth1_new_lag3_4_monthly_avg_category_3_C_mean,auth1_new_lag3_4_monthly_avg_category_1_sum,auth1_new_lag3_4_monthly_avg_category_1_mean,auth1_new_lag3_4_monthly_avg_month_lag_mean,auth1_new_lag3_4_monthly_avg_month_lag_std,auth1_new_lag3_4_monthly_avg_yyyy_week_nunique,auth1_new_lag3_4_monthly_avg_month_diff_mean,auth1_new_lag3_4_monthly_avg_month_diff_std,auth1_new_lag3_4_monthly_avg_purchase_amount_sum,auth1_new_lag3_4_monthly_avg_purchase_amount_max,auth1_new_lag3_4_monthly_avg_purchase_amount_min,auth1_new_lag3_4_monthly_avg_purchase_amount_mean,auth1_new_lag3_4_monthly_avg_installments_sum,auth1_new_lag3_4_monthly_avg_installments_max,auth1_new_lag3_4_monthly_avg_installments_min,auth1_new_lag3_4_monthly_avg_installments_mean,auth1_new_lag3_4_monthly_avg_installments_std,auth1_new_lag3_4_monthly_avg_merchant_id_nunique,auth1_new_lag3_4_monthly_avg_merchant_category_id_nunique,auth1_new_lag3_4_monthly_avg_card_id_size,auth1_new_lag3_4_monthly_avg_city_id_nunique,auth1_new_lag3_4_monthly_avg_state_id_nunique,auth1_new_lag3_4_monthly_avg_subsector_id_nunique
C_ID_00007093c1,0.000000,0.000000,1.000000,0.0,0.000000,0.0,0.750000,0.250000,0.0,0.000000,-8.500000,0.577350,3.0,2017-06-10 11:06:51,2017-05-18 16:54:17,332.750000,10.719919,1.172852,0.512207,0.045074,0.293213,6.0,3.0,1.0,1.500000,1.000000,4.0,4.0,4.0,2.0,1.0,3.0,22.0,0.173913,0.050994,0.260870,0.044444,0.013031,0.066650,0.167603,0.117310,0.000000,0.000000,1.000000,0.0,0.000000,0.0,0.750000,0.250000,0.00,0.000000,-8.5,0.0,1.5,332.750000,4.596194,0.586914,0.481445,0.105164,0.293457,3.000000,2.0,1.000000,1.500000,0.707031,2.0,2.0,2.0,1.50,1.00,1.50
C_ID_0001238066,0.541667,0.000000,0.000000,0.0,0.375000,0.0,0.750000,0.187500,1.0,0.020833,-1.645833,0.668106,12.0,2018-01-25 18:40:51,2017-10-16 20:21:20,123.583333,20.652993,9.851562,1.514648,0.012032,0.205322,83.0,10.0,0.0,1.729492,2.050781,43.0,22.0,48.0,16.0,6.0,15.0,100.0,0.475248,0.097540,0.821782,0.533333,0.109436,0.922363,0.117310,0.075195,0.747024,0.000000,0.000000,0.0,0.205357,0.0,0.867560,0.096726,0.25,0.011905,-2.5,0.0,3.0,149.293155,8.080019,2.462891,0.617188,0.041504,0.158447,20.750000,4.5,0.750000,1.369141,1.252930,11.5,8.0,12.0,5.75,2

100%|██████████| 3/3 [00:00<00:00, 65.23it/s]


(366503, 22)
auth0_new_lag3_4
(325540, 71)


,auth0_new_lag3_4_category_2_1.0_mean,auth0_new_lag3_4_category_2_2.0_mean,auth0_new_lag3_4_category_2_3.0_mean,auth0_new_lag3_4_category_2_4.0_mean,auth0_new_lag3_4_category_2_5.0_mean,auth0_new_lag3_4_category_3_A_mean,auth0_new_lag3_4_category_3_B_mean,auth0_new_lag3_4_category_3_C_mean,auth0_new_lag3_4_category_1_sum,auth0_new_lag3_4_category_1_mean,auth0_new_lag3_4_month_lag_mean,auth0_new_lag3_4_month_lag_std,auth0_new_lag3_4_yyyy_week_nunique,auth0_new_lag3_4_purchase_date_max,auth0_new_lag3_4_purchase_date_min,auth0_new_lag3_4_month_diff_mean,auth0_new_lag3_4_month_diff_std,auth0_new_lag3_4_purchase_amount_sum,auth0_new_lag3_4_purchase_amount_max,auth0_new_lag3_4_purchase_amount_min,auth0_new_lag3_4_purchase_amount_mean,auth0_new_lag3_4_installments_sum,auth0_new_lag3_4_installments_max,auth0_new_lag3_4_installments_min,auth0_new_lag3_4_installments_mean,auth0_new_lag3_4_installments_std,auth0_new_lag3_4_merchant_id_nunique,auth0_new_lag3_4_merchant_category_id_nunique,auth0_new_lag3_4_card_id_size,auth0_new_lag3_4_city_id_nunique,auth0_new_lag3_4_state_id_nunique,auth0_new_lag3_4_subsector_id_nunique,auth0_new_lag3_4_this_term,auth0_new_lag3_4_freq_per_this_term,auth0_new_lag3_4_amount_per_this_term,auth0_new_lag3_4_instthisments_per_this_term,auth0_new_lag3_4_freq_per_90days,auth0_new_lag3_4_amount_per_90days,auth0_new_lag3_4_installments_per_90days,auth0_new_lag3_4_amount_per_installments_sum,auth0_new_lag3_4_amount_per_installments_mean,auth0_new_lag3_4_monthly_avg_category_2_1.0_mean,auth0_new_lag3_4_monthly_avg_category_2_2.0_mean,auth0_new_lag3_4_monthly_avg_category_2_3.0_mean,auth0_new_lag3_4_monthly_avg_category_2_4.0_mean,auth0_new_lag3_4_monthly_avg_category_2_5.0_mean,auth0_new_lag3_4_monthly_avg_category_3_A_mean,auth0_new_lag3_4_monthly_avg_category_3_B_mean,auth0_new_lag3_4_monthly_avg_category_3_C_mean,auth0_new_lag3_4_monthly_avg_category_1_sum,auth0_new_lag3_4_monthly_avg_category_1_mean,auth0_new_lag3_4_monthly_avg_month_lag_mean,auth0_new_lag3_4_monthly_avg_month_lag_std,auth0_new_lag3_4_monthly_avg_yyyy_week_nunique,auth0_new_lag3_4_monthly_avg_month_diff_mean,auth0_new_lag3_4_monthly_avg_month_diff_std,auth0_new_lag3_4_monthly_avg_purchase_amount_sum,auth0_new_lag3_4_monthly_avg_purchase_amount_max,auth0_new_lag3_4_monthly_avg_purchase_amount_min,auth0_new_lag3_4_monthly_avg_purchase_amount_mean,auth0_new_lag3_4_monthly_avg_installments_sum,auth0_new_lag3_4_monthly_avg_installments_max,auth0_new_lag3_4_monthly_avg_installments_min,auth0_new_lag3_4_monthly_avg_installments_mean,auth0_new_lag3_4_monthly_avg_installments_std,auth0_new_lag3_4_monthly_avg_merchant_id_nunique,auth0_new_lag3_4_monthly_avg_merchant_category_id_nunique,auth0_new_lag3_4_monthly_avg_card_id_size,auth0_new_lag3_4_monthly_avg_city_id_nunique,auth0_new_lag3_4_monthly_avg_state_id_nunique,auth0_new_lag3_4_monthly_avg_subsector_id_nunique
C_ID_00007093c1,0.0,0.0,1.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,-8.5,0.547723,2.0,2017-06-17 10:54:36,2017-05-18 16:52:22,332.0,16.431677,4.664062,1.054688,0.450684,0.777344,12.0,3.0,1.0,2.0,1.095703,2.0,2.0,6.0,1.0,1.0,2.0,29.0,0.20,0.155469,0.40,0.066667,0.051819,0.133301,0.358887,0.259033,0.0,0.0,1.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,-8.5,0.0,1.0,332.0,0.000000,2.332031,0.903320,0.601074,0.777344,6.0,2.0,2.0,2.0,0.000000,1.0,1.0,3.0,1.0,1.0,1.0
C_ID_0001238066,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,-2.0,0.000000,1.0,2017-12-30 18:50:35,2017-12-30 18:49:51,121.0,0.000000,0.300537,0.150269,0.150269,0.150269,2.0,1.0,1.0,1.0,0.000000,1.0,1.0,2.0,1.0,1.0,1.0,0.0,2.00,0.300537,2.00,0.022222,0.003340,0.022217,0.100159,0.075134,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,-2.0,0.0,1.0,121.0,0.000000,0.300537,0.150269,0.150269,0.150269,2.0,1.0,1.0,1.0,0.000000,1.0,1.0,2.0,1.0,1.0,1.0
C_ID_0001506ef0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

,auth1_new_lag3_4_category_2_1.0_mean,auth1_new_lag3_4_category_2_2.0_mean,auth1_new_lag3_4_category_2_3.0_mean,auth1_new_lag3_4_category_2_4.0_mean,auth1_new_lag3_4_category_2_5.0_mean,auth1_new_lag3_4_category_3_A_mean,auth1_new_lag3_4_category_3_B_mean,auth1_new_lag3_4_category_3_C_mean,auth1_new_lag3_4_category_1_sum,auth1_new_lag3_4_category_1_mean,auth1_new_lag3_4_month_lag_mean,auth1_new_lag3_4_month_lag_std,auth1_new_lag3_4_yyyy_week_nunique,auth1_new_lag3_4_purchase_date_max,auth1_new_lag3_4_purchase_date_min,auth1_new_lag3_4_month_diff_mean,auth1_new_lag3_4_month_diff_std,auth1_new_lag3_4_purchase_amount_sum,auth1_new_lag3_4_purchase_amount_max,auth1_new_lag3_4_purchase_amount_min,auth1_new_lag3_4_purchase_amount_mean,auth1_new_lag3_4_installments_sum,auth1_new_lag3_4_installments_max,auth1_new_lag3_4_installments_min,auth1_new_lag3_4_installments_mean,auth1_new_lag3_4_installments_std,auth1_new_lag3_4_merchant_id_nunique,auth1_new_lag3_4_merchant_category_id_nunique,auth1_new_lag3_4_card_id_size,auth1_new_lag3_4_city_id_nunique,auth1_new_lag3_4_state_id_nunique,auth1_new_lag3_4_subsector_id_nunique,auth1_new_lag3_4_this_term,auth1_new_lag3_4_freq_per_this_term,auth1_new_lag3_4_amount_per_this_term,auth1_new_lag3_4_instthisments_per_this_term,auth1_new_lag3_4_freq_per_90days,auth1_new_lag3_4_amount_per_90days,auth1_new_lag3_4_installments_per_90days,auth1_new_lag3_4_amount_per_installments_sum,auth1_new_lag3_4_amount_per_installments_mean,auth1_new_lag3_4_monthly_avg_category_2_1.0_mean,auth1_new_lag3_4_monthly_avg_category_2_2.0_mean,auth1_new_lag3_4_monthly_avg_category_2_3.0_mean,auth1_new_lag3_4_monthly_avg_category_2_4.0_mean,auth1_new_lag3_4_monthly_avg_category_2_5.0_mean,auth1_new_lag3_4_monthly_avg_category_3_A_mean,auth1_new_lag3_4_monthly_avg_category_3_B_mean,auth1_new_lag3_4_monthly_avg_category_3_C_mean,auth1_new_lag3_4_monthly_avg_category_1_sum,auth1_new_lag3_4_monthly_avg_category_1_mean,auth1_new_lag3_4_monthly_avg_month_lag_mean,auth1_new_lag3_4_monthly_avg_month_lag_std,auth1_new_lag3_4_monthly_avg_yyyy_week_nunique,auth1_new_lag3_4_monthly_avg_month_diff_mean,auth1_new_lag3_4_monthly_avg_month_diff_std,auth1_new_lag3_4_monthly_avg_purchase_amount_sum,auth1_new_lag3_4_monthly_avg_purchase_amount_max,auth1_new_lag3_4_monthly_avg_purchase_amount_min,auth1_new_lag3_4_monthly_avg_purchase_amount_mean,auth1_new_lag3_4_monthly_avg_installments_sum,auth1_new_lag3_4_monthly_avg_installments_max,auth1_new_lag3_4_monthly_avg_installments_min,auth1_new_lag3_4_monthly_avg_installments_mean,auth1_new_lag3_4_monthly_avg_installments_std,auth1_new_lag3_4_monthly_avg_merchant_id_nunique,auth1_new_lag3_4_monthly_avg_merchant_category_id_nunique,auth1_new_lag3_4_monthly_avg_card_id_size,auth1_new_lag3_4_monthly_avg_city_id_nunique,auth1_new_lag3_4_monthly_avg_state_id_nunique,auth1_new_lag3_4_monthly_avg_subsector_id_nunique,auth0_new_lag3_4_category_2_1.0_mean,auth0_new_lag3_4_category_2_2.0_mean,auth0_new_lag3_4_category_2_3.0_mean,auth0_new_lag3_4_category_2_4.0_mean,auth0_new_lag3_4_category_2_5.0_mean,auth0_new_lag3_4_category_3_A_mean,auth0_new_lag3_4_category_3_B_mean,auth0_new_lag3_4_category_3_C_mean,auth0_new_lag3_4_category_1_sum,auth0_new_lag3_4_category_1_mean,auth0_new_lag3_4_month_lag_mean,auth0_new_lag3_4_month_lag_std,auth0_new_lag3_4_yyyy_week_nunique,auth0_new_lag3_4_purchase_date_max,auth0_new_lag3_4_purchase_date_min,auth0_new_lag3_4_month_diff_mean,auth0_new_lag3_4_month_diff_std,auth0_new_lag3_4_purchase_amount_sum,auth0_new_lag3_4_purchase_amount_max,auth0_new_lag3_4_purchase_amount_min,auth0_new_lag3_4_purchase_amount_mean,auth0_new_lag3_4_installments_sum,auth0_new_lag3_4_installments_max,auth0_new_lag3_4_installments_min,auth0_new_lag3_4_installments_mean,auth0_new_lag3_4_installments_std,auth0_new_lag3_4_merchant_id_nunique,auth0_new_lag3_4_merchant_category_id_nunique,auth0_new_lag3_4_card_id_size,auth0_new_lag3_4_city_id_nunique,auth0_new_lag3_4_state_id_nunique,auth0_new_lag3_4_subsec

In [12]:
#========================================================================
# Save Feature
#========================================================================
fname = f'180_fe4'
ignore_features = ['unix_first_active_month', 'first_active_month', 'card_id', target]

train_test.columns =  [col.replace('.', '_') for col in train_test.columns]

for col in train_test.columns:
    if col in ignore_features: continue
    if (col.count('feature_')):continue
    if (col.count('purchase_date')):continue
    feature = train_test[col].fillna(-1).astype('float32').values
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_{col}@', obj=feature)